In [1]:
ticker = 'aapl'
expiration_date = '2022-06-10' # YYYY-MM-DD

In [2]:
from yahoo_fin import options, stock_info
import pandas as pd
import numpy as np
# gets the data for nearest upcoming expiration date
# options.get_options_chain("aapl")

# specific expiration date
val = options.get_options_chain(ticker, expiration_date)

In [3]:
underlying_price = stock_info.get_live_price(ticker)
underlying_price

147.9600067138672

In [4]:
from typing import Union

def pre_process(option_chain: Union[dict, pd.DataFrame], option_type: str) -> pd.DataFrame:
    """
    Pre-processes the data for the given ticker and expiration date.
    """
    if type(option_chain) is dict:
        df = option_chain[option_type]
    elif type(option_chain) is pd.DataFrame:
        df = option_chain
    else:
        raise TypeError("option_chain must be a dict or pd.DataFrame")
    
    # Contract Name can be derived
    # Trade Date Irrelevant for high liquidity stocks
    # Not Sure what 'Open Interest' is
    df = df.set_index('Strike').drop(['Contract Name', 'Last Trade Date', 'Open Interest'], axis = 1)
    
    # Convert volatility percetage to decimal
    df['Implied Volatility'] = df['Implied Volatility'].str.replace('%','').astype(float)/100
    
    # Convert %change percetage to decimal
    df['% Change'] = df['% Change'].str.replace('%','').replace('-','Nan').astype(float)/100

    # Convert Volume to Integer
    if df['Volume'].dtype == 'object':
        df['Volume'] = df['Volume'].str.replace('-','Nan').astype(float)
    
    # Check
    if option_type == 'calls':
        df.insert(0, 'ITM', np.where(df.index <= underlying_price, 'X', ''))
    elif option_type == 'puts':
        df.insert(0, 'ITM', np.where(df.index >= underlying_price, 'X', ''))


    # Drop pct change, Volume for now?
    # Maybe drop Options with Volume < 1,000?
    df = df[df['Volume'] >= 1000]
    df = df.drop(['% Change'], axis = 1)

    
    return df

# pre_process(val,'calls')

In [5]:
val = { k: pre_process(v, k) for k, v in val.items()}
val = pd.concat(val, axis=1).sort_index().dropna(subset=[('calls','ITM'), ('puts','ITM')])

print(val.shape)
val


(8, 14)


calls                                                         puts  \
         ITM Last Price  Bid  Ask Change   Volume Implied Volatility  ITM   
Strike                                                                      
140.0      X       8.10  0.0  0.0    0.0   1111.0             0.0000        
145.0      X       3.60  0.0  0.0    0.0   7295.0             0.0000        
146.0      X       2.83  0.0  0.0    0.0   4594.0             0.0000        
147.0      X       2.16  0.0  0.0    0.0   8960.0             0.0000        
148.0              1.61  0.0  0.0    0.0  30747.0             0.0020    X   
149.0              1.13  0.0  0.0    0.0  59304.0             0.0313    X   
150.0              0.76  0.0  0.0    0.0  91705.0             0.0625    X   
152.5              0.23  0.0  0.0    0.0  45372.0             0.1250    X   

                                                                
       Last Price  Bid  Ask Change   Volume Implied Volatility  
Strike                                                          
140.0        0.10  0.0  0.0    0.0  13716.0             0.1250  
145.0        0.59  0.0  0.0    0.0  24849.0             0.0625  
146.0        0.85  0.0  0.0    0.0  17966.0             0.0625  
147.0        1.19  0.0  0.0    0.0  24567.0             0.0313  
148.0        1.62  0.0  0.0    0.0  27192.0             0.0000  
149.0        2.18  0.0  0.0    0.0  18874.0             0.0000  
150.0        2.73  0.0  0.0    0.0  15404.0             0.0000  
152.5        4.76  0.0  0.0    0.0   1940.0             0.0000